In [1]:
import logging
import random
from pathlib import Path

from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, select, func, desc, text, label
from gorillatracker.ssl_pipeline.models import TrackingFrameFeature, Tracking, Video

from gorillatracker.ssl_pipeline.dataset import GorillaDataset, SSLDataset
from gorillatracker.ssl_pipeline.feature_mapper import correlate_videos
from gorillatracker.ssl_pipeline.helpers import remove_processed_videos
from gorillatracker.ssl_pipeline.queries import load_processed_videos, load_videos
from gorillatracker.ssl_pipeline.video_preprocessor import preprocess_videos
from gorillatracker.ssl_pipeline.video_processor import multiprocess_predict_and_store, multiprocess_track_and_store
from gorillatracker.ssl_pipeline.visualizer import multiprocess_visualize_video

In [2]:
dataset = GorillaDataset("sqlite:///../test.db")

In [9]:
with Session(dataset.engine) as session:
    videos = load_processed_videos(session, "2024-04-09", [])
    test = session.execute(
        select(Video.fps, func.sum(Video.frames).label("count"))
        .group_by(Video.video_id)
        .order_by(desc("count"))
        )
    
#print(videos)
print(test.all())

[(60, 5295), (60, 2895), (60, 2775), (60, 2535), (60, 1995), (60, 1875), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1815), (60, 1215)]


In [11]:
from sqlalchemy import MetaData,Table, select
metadata = MetaData()

# Reflect the tables from the database
metadata.reflect(bind=dataset.engine)

eng = dataset.engine
session = Session(eng)
print("Tables in the database:")
for table_name in metadata.tables.keys():
    print(f"\nTable: {table_name}")
    table = Table(table_name, metadata, autoload=True, autoload_with=eng)
    select_query = select([table]).limit(5)  # Fetch 5 rows
    result = eng.execute(select_query).fetchall()
    if result:
        print("Table head:")
        print(result)
    else:
        print("Table is empty.")


Tables in the database:

Table: camera


ArgumentError: Column expression, FROM clause, or other columns clause element expected, got [Table('camera', MetaData(), Column('camera_id', INTEGER(), table=<camera>, primary_key=True, nullable=False), Column('name', VARCHAR(length=255), table=<camera>, nullable=False), Column('latitude', FLOAT(), table=<camera>, nullable=False), Column('longitude', FLOAT(), table=<camera>, nullable=False), schema=None)]. Did you mean to say select(Table('camera', MetaData(), Column('camera_id', INTEGER(), table=<camera>, primary_key=True, nullable=False), Column('name', VARCHAR(length=255), table=<camera>, nullable=False), Column('latitude', FLOAT(), table=<camera>, nullable=False), Column('longitude', FLOAT(), table=<camera>, nullable=False), schema=None))?